In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/home/workspace/data/GLDv2/train/train_custom.csv')

In [5]:
df.head()

,id,landmark_id,supercategory,description,language
0,4e7c3c4e083ee269,0,horse racing venue,The Happy Valley Racecourse is one of two race...,en
1,0ddc92c6353c019a,0,horse racing venue,The Happy Valley Racecourse is one of two race...,en
2,d500d704532092dc,0,horse racing venue,The Happy Valley Racecourse is one of two race...,en
3,8a80dbb49ccb966d,0,horse racing venue,The Happy Valley Racecourse is one of two race...,en
4,4a454571caa80175,0,horse racing venue,The Happy Valley Racecourse is one of two race...,en


In [10]:
cut_df = df.iloc[:90:30, :]

In [12]:
cut_df.to_csv('/home/workspace/data/GLDv2/train/train_custom_demo.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2549598 entries, 0 to 2549597
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   id             object
 1   landmark_id    int64 
 2   supercategory  object
 3   description    object
 4   language       object
dtypes: int64(1), object(4)
memory usage: 97.3+ MB


In [5]:
import torch
"cuda" if torch.cuda.is_available() else "cpu"

'cpu'

In [1]:
import torch
torch.cuda.is_available()

True

In [7]:
torch.__version__

'2.8.0+cu129'